In [65]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv

load_dotenv()

True

In [17]:
def get_chatbot_response(client, model_name, messages, temperature=0.0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})
    
    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    ).choices[0].message.content

    return response
    


In [10]:
client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_CHATBOT_URL")
)

model_name = os.getenv("MODEL_NAME")

# Get LLM response

In [22]:
messages = [{"role": "system", "content": "Whats the capital of Australia?"}]
response = get_chatbot_response(client, model_name, messages)

KeyboardInterrupt: 

In [ ]:
response

'The capital of Australia is Canberra.'

# Prompt Engineering Techniques

## Structured Output

In [24]:
system_prompt = """
You are a helpful assistant that answers questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
[
{
    "country": the country you will get the capital of
    "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What's the capital of India?"})
response = get_chatbot_response(client, model_name, messages)
print(response)


[
    {
        "country": "India",
        "capital": "New Delhi"
    }
]


In [27]:
type(response)

str

In [29]:
json_response = json.loads(response)
json_response

[{'country': 'India', 'capital': 'New Delhi'}]

In [32]:
type(json_response[0]),json_response[0]['capital']

(dict, 'New Delhi')

## Input Structuring

In [33]:
user_input = """
Get me the capitals of the following countries:
```
1. Italy
2. Germany
3. France
```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client,model_name,messages)
print(response)

[
    {
        "country": "Italy",
        "capital": "Rome"
    },
    {
        "country": "Germany",
        "capital": "Berlin"
    },
    {
        "country": "France",
        "capital": "Paris"
    }
]


In [34]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

## Give the model time to think (Chain of thought)

In [35]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user", "content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 4
}


In [36]:
259/2*8654+91072*33-12971

4113098.0

In [37]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user", "content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    "result": 12971
}


In [38]:
4113098.0-12971

4100127.0

In [41]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user", "content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

{
    steps: [
        "First, divide 259 by 2 to get 129.5",
        "Then, multiply 129.5 by 8654 to get 111,111.5",
        "Next, multiply 91072 by 33 to get 299,296.4",
        "Finally, subtract 29,971.4 from 299,296.4 to get 269,324.0"
    ],
    result: 269324.0
}


# RAG - Retrieval Augmented Generation

In [44]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)


I can provide you with some of the rumored and confirmed features of the upcoming iPhone 16. Please note that these are subject to change and may not be entirely accurate.

**Rumored Features:**

1. **Improved Cameras:** The iPhone 16 is expected to feature a new 48MP primary camera, with a wider-angle lens and improved low-light performance.
2. **5G and 6G:** The iPhone 16 is rumored to support 5G networks, as well as potential 6G connectivity, which promises even faster data speeds.
3. **Foldable Display:** Apple has filed patents for a foldable iPhone design, which could be a game-changer for the company.
4. **Long-Lasting Battery:** The iPhone 16 is expected to have a longer-lasting battery, possibly with a faster charging system.
5. **Improved Performance:** The iPhone 16 is rumored to feature a faster A17 Bionic chip, which could provide a significant boost in performance.
6. **New Colors:** Apple has confirmed that the iPhone 16 will be available in new colors, including a "Star

In [45]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [46]:
user_prompt = """
{iphone_16}

What's new in iphone 16?
"""

messages = [{"role":"user","content":user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The iPhone 16 is expected to feature several significant upgrades and new technologies. Here are some of the rumored features:

1. **Improved Cameras**: The iPhone 16 is expected to have a new camera system with a wider-angle lens, a telephoto lens, and a new ultra-wide-angle lens. The camera system is also rumored to have improved low-light performance and enhanced portrait mode capabilities.
2. **Faster Processor**: The iPhone 16 is expected to be powered by Apple's A17 Bionic chip, which is said to be faster and more efficient than the A15 Bionic chip in the iPhone 15.
3. **5G and 6G**: The iPhone 16 is expected to support 5G networks, but it's also rumored to have the ability to support 6G networks, which are expected to offer even faster speeds and lower latency.
4. **Long-Lasting Battery**: The iPhone 16 is expected to have a longer-lasting battery, with some rumors suggesting that it could last up to 2 days on a single charge.
5. **New Design**: The iPhone 16 is expected to have

### Automatically Extract context data from database

In [48]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [49]:
data = [iphone_16,samsung_s23]

In [50]:
user_prompt = """What's new in iphone16?"""

In [51]:
embedding_client = OpenAI(
    api_key=os.getenv("RUNPOD_TOKEN"),
    base_url=os.getenv("RUNPOD_EMBEDDING_URL")
)

In [55]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(input=text_input, model=model_name)

    embeddings = []
    for embedding_object in output.data:
        embeddings.append(embedding_object.embedding)
    
    return embeddings

In [61]:
user_prompt_embedding =  get_embedding(embedding_client, model_name, user_prompt)[0]

In [62]:
data_embeddings = [get_embedding(embedding_client, model_name, text)[0] for text in data]

In [66]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

In [67]:
data_similarity

array([[0.73422507, 0.53271344]])

In [68]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

np.int64(0)

In [69]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [70]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [71]:
print(user_prompt_with_data)



The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learnin

In [72]:
messages = [{"role":"user","content":user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)

The iPhone 16 introduces several exciting updates, including:

1. Larger displays:
	* 6.1-inch base model
	* 6.7-inch iPhone 16 Plus
	* 6.3-inch iPhone 16 Pro
	* 6.9-inch iPhone 16 Pro Max
2. Thinner bezels:
	* Ceramic Shield, a more durable and scratch-resistant material
3. Improved performance:
	* New A18 chip (A18 Pro for Pro models)
	* Enhanced neural engine capabilities
	* Faster GPU for gaming
	* Machine learning tasks
4. Upgraded camera systems:
	* Dual-camera setup with a 48MP main sensor (base iPhone 16)
	* 48MP Ultra Wide and 5x telephoto camera (Pro models)
	* "Camera Control" button for more flexible photography options
5. Advanced audio features:
	* "Audio Mix" using machine learning to separate background sounds from speech
6. Extended battery life:
	* Longest-lasting battery of any iPhone 9
7. Faster charging and data transfer:
	* USB-C for faster charging and data transfer
8. Up to 2x faster video encoding:
	* Support for Pro models
9. Consistent starting prices:
	* iPh